In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import numpy as np
import matplotlib.pyplot as plt

import time

In [2]:
# torch.manual_seed(42)

In [3]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(f'device: {device}')

device: cpu


In [4]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')

        self.layers = layers
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)

        self.H1 = self.linears[0]

        
    'forward pass'
    def forward(self,x,y):              
        
        # for i in range(len(self.layers)-2):
        #     z = self.linears[i](a)
        #     a = self.activation(z)

        a = torch.cat([x,y], dim = 1)    #(N,2)

        for i in range(len(self.layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)


            
        b = self.linears[-1](a) 
         
        return b
    
    # def forward_direct(self, x):
        
    #     z = x.float()
    #     H = self.linears[0].weight

    #     for i in range(len(self.layers)-2):
    #         L = self.linears[i](z)
    #         z = self.activation(L)
    #         G = (1-torch.square(z))*H.t() #\sigma'(L)*H
    #         H = torch.matmul(self.linears[i+1].weight,G.t())

    #     z = self.linears[-1](z)
         
    #     return z,H

In [5]:
# layers = np.array([2,50,50,50,50,50,1])
layers = np.array([2,20,20,20,20,20,1])
# PINN = Sequentialmodel(layers).to(device)

In [6]:
# Resetting to ensure the reported peak truly reflects the training loop, rather than including earlier setup.

# if device.type == 'cuda':
#     torch.cuda.reset_peak_memory_stats(device)

In [20]:
# Create the training data

x = torch.linspace(0,1,200, requires_grad = True).view(-1,1)
y = torch.linspace(0,1,200, requires_grad = True).view(-1,1)



if torch.is_tensor(x) != True:         
    x = torch.from_numpy(x)  
if torch.is_tensor(y) != True:         
    y = torch.from_numpy(y) 


#convert to float
x = x.float()
y = y.float()


    
x_train,y_train = torch.meshgrid(x.squeeze(),y.squeeze(), indexing = 'xy')

# x_train = x_train.reshape(-1,1).to(device).requires_grad_(True)     
# y_train = y_train.reshape(-1,1).to(device).requires_grad_(True) 
     

x_train = x_train.reshape(-1,1).requires_grad_(True)     
y_train = y_train.reshape(-1,1).requires_grad_(True) 
     



In [21]:
# Create boundary points

def generate_boundary_points(N = 10000):
    x_0 = torch.zeros(N,1) ; x_1 = torch.ones(N,1); ylr = torch.linspace(0,1.,N).unsqueeze(-1)
    # b_l = torch.cat([x_0,ylr], dim = 1); b_r = torch.cat([x_1,ylr], dim = 1)

    y_0 = torch.zeros(N,1); y_1 = torch.ones(N,1); xbt = torch.linspace(0,1.,N).unsqueeze(-1)
    # b_b = torch.cat([xbt,y_0], dim = 1); b_t = torch.cat([xbt,y_1], dim = 1)

    # x_0 = x_0.to(device)
    # x_1 = x_1.to(device)
    # ylr = ylr.to(device)
    


    # y_0 = y_0.to(device)
    # y_1 = y_1.to(device)
    # xbt = xbt.to(device)

    return x_0, x_1, ylr, y_0, y_1, xbt









In [22]:
def pde_residual(x, y):
    u = PINN(x,y)

    du_dx = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    du_dy = torch.autograd.grad(u, y, torch.ones_like(u), create_graph=True)[0]
    

    du_dx_x = torch.autograd.grad(du_dx, x, torch.ones_like(du_dx), create_graph=True)[0]
    du_dy_y = torch.autograd.grad(du_dy, y, torch.ones_like(du_dy), create_graph=True)[0]
    

    res_pde = du_dx_x + du_dy_y

    return res_pde


    

In [23]:
def boundary_condition(x,y):

    x_0, x_1, ylr, y_0, y_1, xbt = generate_boundary_points()

    u_x_0 = PINN(x_0, ylr)
    u_x_1 = PINN(x_1,ylr)

    u_y_0 = PINN(xbt,y_0)
    u_y_1 = PINN(xbt,y_1)

   

    res_x_0 = u_x_0 - torch.zeros_like(ylr)
    res_x_1 = u_x_1 - torch.sin(np.pi * ylr)
    res_y_0 = u_y_0 - torch.zeros_like(xbt)
    res_y_1 = u_y_1 - torch.zeros_like(xbt)
    

    return res_x_0,res_x_1,res_y_0,res_y_1

In [24]:
def compute_losses():
   res_pde = pde_residual(x_train, y_train) 
   
   res_x_0,res_x_1,res_y_0,res_y_1 = boundary_condition(x_train, y_train)

   loss_pde = torch.mean(res_pde**2)
   
   loss_bc = torch.mean(res_x_0**2) + torch.mean(res_x_1**2) + torch.mean(res_y_0**2) + torch.mean(res_y_1**2)

   total_loss = loss_pde + loss_bc

   return total_loss

In [ ]:
# optimizer = torch.optim.Adam(PINN.parameters(), lr=0.01)

In [ ]:
# No. of epochs


# start_time = time.time()

# num_epochs = 10000



# for epoch in range(num_epochs):
#     optimizer.zero_grad()

#     total_loss = compute_losses()

    
#     total_loss.backward()

#     optimizer.step()

#     if (epoch) % 200 == 0:
#      print(f'Epoch {epoch}, Loss: {total_loss.item()}')


# end_time = time.time()

# print(f'Total Training Time: {(end_time - start_time): .4f}seconds')


    








In [25]:
torch.manual_seed(42)
# PINN = Sequentialmodel(layers).to(device)
PINN = Sequentialmodel(layers)

In [ ]:
# if device.type == 'cuda':
#     torch.cuda.reset_peak_memory_stats(device)

In [ ]:
# optimizer = torch.optim.Adam(PINN.parameters(), lr=0.01)

In [ ]:
# # Threshold loss as the stopping criteria

# max_epochs = 15000
# threshold = 0.002



# start_time = time.time()

# ep = 0
# while ep < max_epochs:
#     optimizer.zero_grad()

#     total_loss = compute_losses()

    
#     total_loss.backward()

#     optimizer.step()


#     if total_loss.item() < threshold:
#         print(f"Reached threshold loss {threshold} at epoch {ep}")
#         break

#     if (ep) % 200 == 0:
#      print(f'Epoch {ep}, Loss: {total_loss.item()}')

#     ep += 1


# print(f"Training stopped at epoch {ep}, total time {time.time() - start_time:.2f} s")





In [26]:
# Using LBFGS

optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.05,max_iter=20,history_size=50,tolerance_grad=1e-9,tolerance_change=1e-9,line_search_fn='strong_wolfe')

max_outer_steps = 15000
threshold = 0.002

start_time = time.time()
ep = 0


def closure():

    optimizer.zero_grad()
    total_loss = compute_losses()
    total_loss.backward()

    return total_loss

while ep < max_outer_steps:

    total_loss = optimizer.step(closure)

    if total_loss.item() < threshold:
        print(f"Reached threshold loss {threshold} at outer step {ep}")
        break

    if ep % 200 == 0:
        print(f'Outer {ep}, Loss: {total_loss.item()}')

    ep += 1

print(f"Training stopped at outer step {ep}, total time {time.time() - start_time:.2f} s")
    

Outer 0, Loss: 0.8001270294189453
Reached threshold loss 0.002 at outer step 30
Training stopped at outer step 30, total time 64.79 s


In [13]:
# # Memory usage after training

# if device.type == 'cuda':
#     peak_mem = torch.cuda.max_memory_allocated(device)
#     print(f'Peak GPU Memory Usage: {peak_mem / 1e6: .2f} MB')